In [1]:
print("ok")

ok


In [2]:
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
# from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PP
from langchain.vectorstores import FAISS  # or another concrete retriever class
from langchain.chains import RetrievalQA

from langchain.embeddings import OpenAIEmbeddings  # or another embedding model
from langchain.docstore import InMemoryDocstore
from langchain.chains import RetrievalQA


c:\Users\abhis\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECOME_API_KEY = '8cf7ae00-94e5-429a-840b-6f7331c15e04'
PINECOME_API_ENV = 'us-east-1'

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf", 
                    loader_cls = PyPDFLoader)
            
    documents = loader.load()
    return documents


In [5]:
extracted_data = load_pdf("data/")

In [6]:
# extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [9]:

# text_chunks

In [10]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

c:\Users\abhis\anaconda3\envs\mchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [13]:
query_result = embeddings.embed_query("hello workd")
print("Length", len(query_result))


Length 384


In [14]:
# query_result

In [15]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY','7281e437-7288-457d-9f17-2724b37a6180')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV','us-east-1')
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY


In [16]:
# # Initializing the Pinecone
# pinecone.init(api_key=PINECOME_API_KEY,
#               environment=PINECOME_API_ENV)

In [17]:
from langchain_pinecone import PineconeVectorStore

import pinecone
pc = PP(api_key =PINECOME_API_KEY)
index_name ="mdi-chatbot"

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

KeyboardInterrupt: 

In [18]:


docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k=3)

print("Result",docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [19]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else.
helpful answer:
"""

In [28]:
PROMPT = PromptTemplate (template=prompt_template, input_variables=["context","question"]
)
chain_type_kwargs={"prompt": PROMPT}
print(chain_type_kwargs)

{'prompt': PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="\nUse the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext:{context}\nQuestion:{question}\n\nOnly return the helpful answer below and nothing else.\nhelpful answer:\n", template_format='f-string', validate_template=True)}


In [29]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [30]:
def retrieval_qa_chain():
      # Replace with the correct instantiation of your retriever

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=docsearch.as_retriever(search_kwargs={'k':2}),
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs
    )
    return qa_chain

In [31]:
# # Create the retrieval QA chain
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
#     return_source_documents=True,
#     chain_type_kwargs=chain_type_kwargs
# )

In [32]:

qa_chain = retrieval_qa_chain()
while True:
    user_input = input(f"Input Prompt:")
    result = qa_chain({"query": user_input})
    print("Response :", result["result"])

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)